In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
import nltk
nltk.download('punkt_tab')
  

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\samam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [132]:
 # load dataset
df = pd.read_csv('datasets/amazon_alexa.tsv' ,sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [133]:
# explore
df.duplicated().sum()

715

In [134]:
df.isna().sum()

rating              0
date                0
variation           0
verified_reviews    1
feedback            0
dtype: int64

In [135]:
df.value_counts()

rating  date       variation    verified_reviews                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [136]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [137]:
# preprocessing
## Lowercase
df['verified_reviews'] = df['verified_reviews'].str.lower()
## remove special characters
df['verified_reviews'] = df['verified_reviews'].apply(lambda x : re.sub('[^a-zA-Z]', ' ', x))
## tokenization
tokens = df['verified_reviews'].apply(lambda x : word_tokenize(x))
## remove stop words
stop_words = set(stopwords.words('english'))
filtered_tokens = tokens.apply(lambda x : [word for word in x if word not in stop_words])
## stemming (porter stemmer) | lemmatization
stemmer = PorterStemmer()
stemmed_tokens = filtered_tokens.apply(lambda x : [stemmer.stem(word) for word in x])

In [138]:
x = stemmed_tokens.apply(lambda x : ' '.join(x))
y = df['feedback']

In [139]:
vectorize = CountVectorizer()
x = vectorize.fit_transform(x)
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [140]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [141]:
y_train.value_counts()

feedback
1    1775
0     172
Name: count, dtype: int64

In [142]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [143]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.893223819301848


In [144]:
### pickle
import pickle
pickle.dump(model,open('model.pkl','wb'))
pickle.dump(vectorize,open('vectorizer.pkl','wb'))